In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Creación de datos con outliers intencionados

In [ ]:
np.random.seed(42)
n_registros = 300

# Generar datos normales
df = pd.DataFrame({
    'paciente_id': [f'P{i:03d}' for i in range(1, n_registros + 1)],
    'HR': np.random.normal(75, 12, n_registros),
    'SBP': np.random.normal(120, 15, n_registros),
    'DBP': np.random.normal(80, 10, n_registros),
    'SpO2': np.random.normal(97, 1.5, n_registros)
})

# Insertar outliers intencionalmente
outlier_indices = [10, 25, 50, 75, 100, 150, 200]
df.loc[10, 'HR'] = 185  # Taquicardia extrema
df.loc[25, 'HR'] = 35   # Bradicardia extrema
df.loc[50, 'SBP'] = 210 # Hipertensión severa
df.loc[75, 'SBP'] = 65  # Hipotensión severa
df.loc[100, 'SpO2'] = 82 # Hipoxemia severa
df.loc[150, 'SpO2'] = 75 # Hipoxemia crítica
df.loc[200, 'DBP'] = 130 # DBP anormal

print("=== Dataset con outliers ===")
print(df.head(15))
print(f"\nTotal de registros: {len(df)}")
print(f"\nEstadísticas básicas:")
print(df[['HR', 'SBP', 'DBP', 'SpO2']].describe())

## 1. Método IQR (Rango Intercuartílico)

In [ ]:
def detectar_outliers_iqr(df, columna):
    """
    Detecta outliers usando el método IQR
    Outliers: valores fuera del rango [Q1 - 1.5*IQR, Q3 + 1.5*IQR]
    """
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    # Identificar outliers
    outliers = (df[columna] < limite_inferior) | (df[columna] > limite_superior)
    
    return outliers, limite_inferior, limite_superior, Q1, Q3, IQR

# Aplicar a HR
print("=== Detección de outliers en HR usando IQR ===")
outliers_hr, lim_inf_hr, lim_sup_hr, q1_hr, q3_hr, iqr_hr = detectar_outliers_iqr(df, 'HR')

print(f"Q1 (percentil 25): {q1_hr:.2f}")
print(f"Q3 (percentil 75): {q3_hr:.2f}")
print(f"IQR: {iqr_hr:.2f}")
print(f"Límite inferior: {lim_inf_hr:.2f}")
print(f"Límite superior: {lim_sup_hr:.2f}")
print(f"\nOutliers detectados: {outliers_hr.sum()}")

if outliers_hr.sum() > 0:
    print("\nRegistros con outliers en HR:")
    print(df[outliers_hr][['paciente_id', 'HR']])

In [ ]:
# Aplicar IQR a todos los signos vitales
print("=== Resumen de outliers por IQR ===")

for columna in ['HR', 'SBP', 'DBP', 'SpO2']:
    outliers, lim_inf, lim_sup, q1, q3, iqr = detectar_outliers_iqr(df, columna)
    print(f"\n{columna}:")
    print(f"  Rango normal: [{lim_inf:.2f}, {lim_sup:.2f}]")
    print(f"  Outliers: {outliers.sum()} ({outliers.sum()/len(df)*100:.2f}%)")
    
    # Marcar en el DataFrame
    df[f'{columna}_outlier_iqr'] = outliers

## 2. Método Z-score

In [ ]:
def detectar_outliers_zscore(df, columna, umbral=3):
    """
    Detecta outliers usando Z-score
    Z-score = (valor - media) / desviación estándar
    Outliers: |Z-score| > umbral (típicamente 3)
    """
    media = df[columna].mean()
    std = df[columna].std()
    
    # Calcular Z-score
    z_scores = (df[columna] - media) / std
    
    # Identificar outliers
    outliers = np.abs(z_scores) > umbral
    
    return outliers, z_scores, media, std

# Aplicar a HR
print("=== Detección de outliers en HR usando Z-score ===")
outliers_hr_z, z_scores_hr, media_hr, std_hr = detectar_outliers_zscore(df, 'HR', umbral=3)

print(f"Media: {media_hr:.2f}")
print(f"Desviación estándar: {std_hr:.2f}")
print(f"Umbral Z-score: ±3")
print(f"Outliers detectados: {outliers_hr_z.sum()}")

if outliers_hr_z.sum() > 0:
    df['HR_zscore'] = z_scores_hr
    print("\nRegistros con outliers (Z-score):")
    print(df[outliers_hr_z][['paciente_id', 'HR', 'HR_zscore']])

In [ ]:
# Aplicar Z-score a todos los signos vitales
print("=== Resumen de outliers por Z-score ===")

for columna in ['HR', 'SBP', 'DBP', 'SpO2']:
    outliers, z_scores, media, std = detectar_outliers_zscore(df, columna, umbral=3)
    print(f"\n{columna}:")
    print(f"  Media ± 3σ: {media:.2f} ± {3*std:.2f}")
    print(f"  Rango: [{media - 3*std:.2f}, {media + 3*std:.2f}]")
    print(f"  Outliers: {outliers.sum()} ({outliers.sum()/len(df)*100:.2f}%)")
    
    # Marcar en el DataFrame
    df[f'{columna}_outlier_zscore'] = outliers
    df[f'{columna}_zscore'] = z_scores

## 3. Método de Percentiles

In [ ]:
def detectar_outliers_percentiles(df, columna, percentil_bajo=1, percentil_alto=99):
    """
    Detecta outliers usando percentiles
    Outliers: valores fuera del rango [P_bajo, P_alto]
    """
    limite_inferior = df[columna].quantile(percentil_bajo / 100)
    limite_superior = df[columna].quantile(percentil_alto / 100)
    
    # Identificar outliers
    outliers = (df[columna] < limite_inferior) | (df[columna] > limite_superior)
    
    return outliers, limite_inferior, limite_superior

print("=== Detección de outliers usando Percentiles (1% y 99%) ===")

for columna in ['HR', 'SBP', 'DBP', 'SpO2']:
    outliers, lim_inf, lim_sup = detectar_outliers_percentiles(df, columna, 1, 99)
    print(f"\n{columna}:")
    print(f"  Percentil 1%: {lim_inf:.2f}")
    print(f"  Percentil 99%: {lim_sup:.2f}")
    print(f"  Outliers: {outliers.sum()} ({outliers.sum()/len(df)*100:.2f}%)")
    
    df[f'{columna}_outlier_percentil'] = outliers

## 4. Umbrales clínicos específicos

In [ ]:
# Definir umbrales clínicos basados en conocimiento médico
print("=== Detección usando umbrales clínicos ===")

# Heart Rate
df['HR_outlier_clinico'] = (df['HR'] < 40) | (df['HR'] > 180)
hr_anormales = df[df['HR_outlier_clinico']]
print(f"\nHR anormal (< 40 o > 180): {len(hr_anormales)}")
if len(hr_anormales) > 0:
    print(hr_anormales[['paciente_id', 'HR']])

# Systolic Blood Pressure
df['SBP_outlier_clinico'] = (df['SBP'] < 70) | (df['SBP'] > 200)
sbp_anormales = df[df['SBP_outlier_clinico']]
print(f"\nSBP anormal (< 70 o > 200): {len(sbp_anormales)}")
if len(sbp_anormales) > 0:
    print(sbp_anormales[['paciente_id', 'SBP']])

# SpO2
df['SpO2_outlier_clinico'] = df['SpO2'] < 85
spo2_anormales = df[df['SpO2_outlier_clinico']]
print(f"\nSpO2 anormal (< 85%): {len(spo2_anormales)}")
if len(spo2_anormales) > 0:
    print(spo2_anormales[['paciente_id', 'SpO2']])

# DBP
df['DBP_outlier_clinico'] = (df['DBP'] < 40) | (df['DBP'] > 120)
dbp_anormales = df[df['DBP_outlier_clinico']]
print(f"\nDBP anormal (< 40 o > 120): {len(dbp_anormales)}")
if len(dbp_anormales) > 0:
    print(dbp_anormales[['paciente_id', 'DBP']])

## 5. Comparación de métodos

In [ ]:
# Comparar resultados de los diferentes métodos
print("=== Comparación de métodos de detección ===")

for columna in ['HR', 'SBP', 'DBP', 'SpO2']:
    print(f"\n{columna}:")
    print(f"  IQR: {df[f'{columna}_outlier_iqr'].sum()} outliers")
    print(f"  Z-score: {df[f'{columna}_outlier_zscore'].sum()} outliers")
    print(f"  Percentiles: {df[f'{columna}_outlier_percentil'].sum()} outliers")
    print(f"  Clínico: {df[f'{columna}_outlier_clinico'].sum()} outliers")
    
    # Outliers detectados por todos los métodos
    todos_metodos = (
        df[f'{columna}_outlier_iqr'] & 
        df[f'{columna}_outlier_zscore'] & 
        df[f'{columna}_outlier_percentil']
    )
    print(f"  Detectados por todos: {todos_metodos.sum()} outliers")

## 6. Visualización de outliers - Boxplot

In [ ]:
# Boxplots para visualizar outliers
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

signos = ['HR', 'SBP', 'DBP', 'SpO2']
titulos = ['Heart Rate (bpm)', 'Systolic BP (mmHg)', 'Diastolic BP (mmHg)', 'SpO2 (%)']

for idx, (signo, titulo) in enumerate(zip(signos, titulos)):
    ax = axes[idx // 2, idx % 2]
    
    # Boxplot
    bp = ax.boxplot(df[signo], vert=True, patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    
    # Marcar outliers detectados por IQR
    outliers_data = df[df[f'{signo}_outlier_iqr']][signo]
    if len(outliers_data) > 0:
        ax.scatter([1] * len(outliers_data), outliers_data, 
                   color='red', s=100, zorder=3, label='Outliers')
    
    ax.set_title(titulo, fontweight='bold')
    ax.set_ylabel('Valor')
    ax.grid(True, alpha=0.3)
    if len(outliers_data) > 0:
        ax.legend()

plt.tight_layout()
plt.show()

print("Boxplots generados (método IQR)")

## 7. Visualización con histogramas

In [ ]:
# Histogramas con outliers marcados
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, (signo, titulo) in enumerate(zip(signos, titulos)):
    ax = axes[idx // 2, idx % 2]
    
    # Histograma de todos los datos
    ax.hist(df[signo], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    
    # Marcar outliers
    outliers_data = df[df[f'{signo}_outlier_zscore']][signo]
    if len(outliers_data) > 0:
        ax.hist(outliers_data, bins=30, alpha=0.9, color='red', 
                edgecolor='darkred', label='Outliers (Z-score)')
    
    # Líneas de media y límites
    media = df[signo].mean()
    std = df[signo].std()
    ax.axvline(media, color='green', linestyle='--', linewidth=2, label='Media')
    ax.axvline(media + 3*std, color='orange', linestyle='--', linewidth=1.5, label='±3σ')
    ax.axvline(media - 3*std, color='orange', linestyle='--', linewidth=1.5)
    
    ax.set_title(titulo, fontweight='bold')
    ax.set_xlabel('Valor')
    ax.set_ylabel('Frecuencia')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Histogramas con outliers generados")

## 8. Reporte de outliers

In [ ]:
# Crear reporte completo de outliers
print("=== REPORTE COMPLETO DE OUTLIERS ===")

# Identificar registros con cualquier outlier
tiene_outlier = (
    df['HR_outlier_clinico'] | 
    df['SBP_outlier_clinico'] | 
    df['DBP_outlier_clinico'] | 
    df['SpO2_outlier_clinico']
)

registros_con_outliers = df[tiene_outlier]

print(f"\nTotal de registros con outliers clínicos: {len(registros_con_outliers)}")
print(f"Porcentaje del dataset: {len(registros_con_outliers)/len(df)*100:.2f}%")

print("\nDetalle de registros anormales:")
columnas_reporte = ['paciente_id', 'HR', 'SBP', 'DBP', 'SpO2',
                     'HR_outlier_clinico', 'SBP_outlier_clinico', 
                     'DBP_outlier_clinico', 'SpO2_outlier_clinico']
print(registros_con_outliers[columnas_reporte])

## 9. Clasificación de severidad

In [ ]:
# Clasificar outliers por severidad
def clasificar_severidad_hr(hr):
    if hr < 40:
        return 'Bradicardia severa'
    elif hr > 180:
        return 'Taquicardia severa'
    elif hr < 50:
        return 'Bradicardia moderada'
    elif hr > 150:
        return 'Taquicardia moderada'
    else:
        return 'Normal'

def clasificar_severidad_spo2(spo2):
    if spo2 < 80:
        return 'Hipoxemia crítica'
    elif spo2 < 85:
        return 'Hipoxemia severa'
    elif spo2 < 90:
        return 'Hipoxemia moderada'
    elif spo2 < 95:
        return 'Hipoxemia leve'
    else:
        return 'Normal'

df['HR_severidad'] = df['HR'].apply(clasificar_severidad_hr)
df['SpO2_severidad'] = df['SpO2'].apply(clasificar_severidad_spo2)

print("=== Distribución de severidad ===")
print("\nHeart Rate:")
print(df['HR_severidad'].value_counts())
print("\nSpO2:")
print(df['SpO2_severidad'].value_counts())

# Casos críticos
casos_criticos = df[
    (df['HR_severidad'].str.contains('severa')) | 
    (df['SpO2_severidad'].str.contains('crítica|severa'))
]

print("\n=== Casos que requieren atención inmediata ===")
print(casos_criticos[['paciente_id', 'HR', 'HR_severidad', 'SpO2', 'SpO2_severidad']])

## 10. Recomendaciones para manejo de outliers

In [ ]:
# Análisis de outliers potencialmente erróneos vs clínicamente significativos
print("=== Análisis de validez de outliers ===")

# Outliers extremos que probablemente son errores de medición
errores_medicion = df[
    (df['HR'] < 30) | (df['HR'] > 200) |  # Biológicamente improbable
    (df['SpO2'] < 70) |  # Incompatible con vida consciente
    (df['SBP'] < 60) | (df['SBP'] > 220)  # Extremos poco probables
]

print(f"Outliers probablemente erróneos: {len(errores_medicion)}")
if len(errores_medicion) > 0:
    print("\nDetalle:")
    print(errores_medicion[['paciente_id', 'HR', 'SBP', 'SpO2']])
    print("\n⚠️ RECOMENDACIÓN: Verificar estos valores con el equipo médico")

# Outliers clínicamente significativos pero plausibles
outliers_clinicos = df[
    tiene_outlier & ~df.index.isin(errores_medicion.index)
]

print(f"\nOutliers clínicamente significativos: {len(outliers_clinicos)}")
if len(outliers_clinicos) > 0:
    print("\nDetalle:")
    print(outliers_clinicos[['paciente_id', 'HR', 'SBP', 'DBP', 'SpO2']].head(10))
    print("\n✓ RECOMENDACIÓN: Estos valores requieren seguimiento clínico")

## Resumen

### Métodos de detección de outliers:

1. **IQR (Rango Intercuartílico)**:
   - Robusto a valores extremos
   - Outliers: fuera de [Q1 - 1.5×IQR, Q3 + 1.5×IQR]
   - Ideal para distribuciones asimétricas

2. **Z-score**:
   - Basado en desviaciones estándar
   - Outliers: |Z| > 3 (típicamente)
   - Sensible a valores extremos
   - Mejor para distribuciones normales

3. **Percentiles**:
   - Flexible y personalizable
   - Outliers: fuera de percentiles definidos (ej: 1% y 99%)
   - Útil cuando se conoce distribución esperada

4. **Umbrales clínicos**:
   - Basado en conocimiento médico
   - Más relevante clínicamente
   - Específico para cada parámetro

### Aplicación clínica:
- **HR > 180 o < 40**: Requiere atención inmediata
- **SpO₂ < 85%**: Hipoxemia severa
- **SBP > 200 o < 70**: Crisis hipertensiva o hipotensión severa

### Buenas prácticas:
- Combinar múltiples métodos para validación
- Distinguir entre errores de medición y eventos clínicos reales
- Visualizar outliers para mejor comprensión
- No eliminar outliers automáticamente sin validación clínica
- Documentar criterios de detección utilizados